In [1]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from keras.optimizers import SGD
import numpy as np
import pickle
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

# dataset load
with open('dataset3D.pickle', 'rb') as f:
    dataset_load = pickle.load(f)
x_train = dataset_load['x_train']
t_train = dataset_load['t_train']
x_test = dataset_load['x_test']
t_test = dataset_load['t_test']

# 이미지 데이터 크기 (28, 28, 1)로 reshape
x_train = x_train.reshape(len(x_train), 28, 28, 1)
x_test = x_test.reshape(len(x_test), 28, 28, 1)
# 픽셀값을 0~1 범위로 정규화
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

ModuleNotFoundError: No module named 'keras'

In [ ]:
def create_model(optimizer, 
                 learn_rate, 
                 init_mode_1, init_mode_2, init_mode_3, init_mode_4,
                 neurons_1, neurons_2, neurons_3,
                 pool_size,
                 padding,
                 dropout_1, dropout_2):
    model = Sequential()
    model.add(Conv2D(neurons_1, (5, 5), padding=padding, kernel_initializer=init_mode_1, activation='relu', input_shape=(28, 28, 1)))
    model.add(Conv2D(neurons_2, (5, 5), padding=padding, kernel_initializer=init_mode_2, activation='relu'))
    model.add(MaxPooling2D(pool_size=pool_size))
    model.add(Dropout(dropout_1))
    model.add(Flatten())
    model.add(Dense(neurons_3, kernel_initializer=init_mode_3, activation='relu'))
    model.add(Dropout(dropout_2))
    model.add(Dense(10, kernel_initializer=init_mode_4, activation='softmax'))
    
    model.compile(metrics=['acc'],
                  optimizer='SGD',
                  loss="sparse_categorical_crossentropy")
    
    return model

# 재현성을 위해 random seed 고정
seed = 0
np.random.seed(seed)

# 아래 param 모든 조합 test
param_grid = {'epochs': [15, 20],
              'batch_size': [60],
              'optimizer' : ['Adadelta'],
              'learn_rate': [0.001],
              'init_mode_1' : ['he_normal'],
              'init_mode_2' : ['he_normal'],
              'init_mode_3' : ['he_normal'],
              'init_mode_4' : ['he_uniform'],
              'neurons_1': [16],
              'neurons_2': [64],
              'neurons_3': [64],
              'pool_size' : [[2,2]],
              'padding' : ['same'],
              'dropout_1' : [0.0],
              'dropout_2' : [0.1]
              }

my_classifier = KerasClassifier(create_model)

grid = GridSearchCV(my_classifier, param_grid, cv=3, n_jobs=1, verbose=1)

grid_result = grid.fit(x_train, t_train)

# 결과 요약
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']

for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))
    
# 결과 출력
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

# Test accuracy 출력
print("Test Accuracy", grid_result.score(x_test, t_test))